# Feature 선택

## 선정 기준
1. EDA 결과 상관계수가 높고 시각화에서 유의미한 결과가 나온 feature로 1차 선택</br>
2. 일반적으로 예상할 수 있는 결과를 나타낸 feature 위주로 선택</br>
    ex) 나이가 적은 고객들이 이탈율이 높다고 생각함 => Age 변수는 나이가 많아질수록 이탈율 높아짐.(선택X)</br>
    Under 30 변수는 30세 이하 고객의 이탈율이 높았음.(선택O)</br> 
3. 같은 성격을 가진 feature 중에서 시각화 결과가 더 뚜렷한 feature 선택</br>
4. 한국 통신사 시장과 맞지 않는 feature 제외</br>
    ex) Contract-알뜰폰은 약정없이 월 청구, 통신3사는 무조건 1년 or 2년 약정)</br>
5. 그 외 중요하다고 생각하는 feature 선정

## 선정 결과&선정 기준
* Dependents (3)
* under 30 (2)
* Monthly Charge (2)
* Total Revenue (2)

* Tenure in Months (5)
* Internet Service (5) : 예상과 다른 결과지만 중요하다고 생각하여 선정
* Device Protection Plan (2)
* Unlimited Data (2)
* Streaming Movies (5)

## 파생변수

* 가입기간(일)
* 가입일자(가입기간으로 계산)</br>

=> 파생변수 생성 과정은 01. 이탈 날짜 예측 참고

# 데이터 로드

In [1]:
%config Completer.use_jedi = False
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta

%matplotlib inline

In [2]:
# 앞서 생성한 Contract Date, Contract Period변수가 있는 데이터  불러오기

df = pd.read_csv("new_total.csv")
print(df.shape)
df.head()

(7043, 14)


,Customer ID,Dependents,Under 30,Monthly Charge,Total Revenue,Tenure in Months,Internet Service,Device Protection Plan,Unlimited Data,Streaming Movies,Contract Date,Contract Period,Churn Label,Churn Score
0,8779-QRDMV,0,0,39.65,59.65,1,1,1,0,1,2023-08-14,47,1,91
1,7495-OOKFY,1,0,80.65,1024.10,8,1,0,1,0,2023-02-08,234,1,69
2,1658-BYGOY,1,0,95.45,1910.88,18,1,0,1,1,2022-04-27,521,1,81
3,4598-XLKNJ,1,0,98.50,2995.07,25,1,1,1,1,2021-09-05,755,1,88
4,4846-WHAFZ,1,0,76.50,3102.36,37,1,0,1,0,2020-09-28,1097,1,67


# 이 고객은 언제 이탈할까?
이탈한 고객의 데이터만 가지고 학습하여 이탈하지 않은 고객 데이터를 가지고 예측</br>

* label = Contract Period

Forecast Period를 예측하기 위해 이탈한 고객의 계약기간으로 학습하여 아직 이탈하지 않은 고객의 이탈 예측 날짜(가입일자+예측한 계약기간)를 예측하려고 함.

In [3]:
# 이탈한 고객의 계약기간으로 학습

train_df = df.loc[df['Churn Label'] == 1]
train_df

,Customer ID,Dependents,Under 30,Monthly Charge,Total Revenue,Tenure in Months,Internet Service,Device Protection Plan,Unlimited Data,Streaming Movies,Contract Date,Contract Period,Churn Label,Churn Score
0,8779-QRDMV,0,0,39.65,59.65,1,1,1,0,1,2023-08-14,47,1,91
1,7495-OOKFY,1,0,80.65,1024.10,8,1,0,1,0,2023-02-08,234,1,69
2,1658-BYGOY,1,0,95.45,1910.88,18,1,0,1,1,2022-04-27,521,1,81
3,4598-XLKNJ,1,0,98.50,2995.07,25,1,1,1,1,2021-09-05,755,1,88
4,4846-WHAFZ,1,0,76.50,3102.36,37,1,0,1,0,2020-09-28,1097,1,67
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2530,5482-NUPNA,0,1,60.40,344.27,4,1,0,1,0,2023-06-24,98,1,90
2531,1122-JWTJW,1,0,70.65,108.33,1,1,0,1,0,2023-08-08,53,1,82
2532,1699-HPSBG,0,0,59.80,843.00,12,1,0,1,0,2022-10-04,361,1,69
2533,8775-CEBBJ,0,0,44.20,780.63,9,1,0,1,0,2023-01-01,272,1,87


In [37]:
# 필요한 컬럼만 가져오기
# 문자형 데이터 Customer ID 제외, label의 종속변수가 될 수 있는 Tenure in Months 제외

# col1=train_df.columns[1:5].to_list()
# col2=train_df.columns[6:10].to_list()
# col3=train_df.columns[11:12].to_list()

# new_col=col1+col2+col3

new_col = ['Dependents', 'Under 30', 'Monthly Charge','Total Revenue', 'Internet Service', 'Device Protection Plan',
           'Unlimited Data', 'Contract Period', 'Churn Score' ]
new_train_df = train_df[new_col]
new_train_df

,Dependents,Under 30,Monthly Charge,Total Revenue,Internet Service,Device Protection Plan,Unlimited Data,Contract Period,Churn Score
0,0,0,39.65,59.65,1,1,0,47,91
1,1,0,80.65,1024.10,1,0,1,234,69
2,1,0,95.45,1910.88,1,0,1,521,81
3,1,0,98.50,2995.07,1,1,1,755,88
4,1,0,76.50,3102.36,1,0,1,1097,67
...,...,...,...,...,...,...,...,...,...
2530,0,1,60.40,344.27,1,0,1,98,90
2531,1,0,70.65,108.33,1,0,1,53,82
2532,0,0,59.80,843.00,1,0,1,361,69
2533,0,0,44.20,780.63,1,0,1,272,87


* 정규화

In [38]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler

scaled_df = new_train_df.copy()

scaled_cols = ['Monthly Charge', 'Total Revenue',  'Churn Score' ]

# MinMaxScaler를 사용한 정규화
scaler = MinMaxScaler()
scaled_df[scaled_cols] = scaler.fit_transform(scaled_df[scaled_cols])
scaled_df

,Dependents,Under 30,Monthly Charge,Total Revenue,Internet Service,Device Protection Plan,Unlimited Data,Contract Period,Churn Score
0,0,0,0.209045,0.003404,1,1,0,47,0.838710
1,1,0,0.621106,0.089718,1,0,1,234,0.129032
2,1,0,0.769849,0.169080,1,0,1,521,0.516129
3,1,0,0.800503,0.266109,1,1,1,755,0.741935
4,1,0,0.579397,0.275711,1,0,1,1097,0.064516
...,...,...,...,...,...,...,...,...,...
2530,0,1,0.417588,0.028876,1,0,1,98,0.806452
2531,1,0,0.520603,0.007761,1,0,1,53,0.548387
2532,0,0,0.411558,0.073510,1,0,1,361,0.129032
2533,0,0,0.254774,0.067928,1,0,1,272,0.709677


In [63]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

num_cols = ['Monthly Charge', 'Total Revenue',  'Churn Score' ]

vif = pd.DataFrame()
vif['VIF Factor'] = [variance_inflation_factor(
    scaled_df[num_cols].values, i) for i in range(scaled_df[num_cols].shape[1])]
vif['features'] = scaled_df[num_cols].columns
print(vif)

   VIF Factor        features
0    4.229867  Monthly Charge
1    2.214076   Total Revenue
2    2.823923     Churn Score


## 학습/테스트셋 분리

In [41]:
from sklearn.model_selection import train_test_split

y_target = scaled_df['Contract Period']
X_data = scaled_df.drop(['Contract Period'],axis=1,inplace=False)

X_train , X_test , y_train , y_test = train_test_split(X_data , y_target ,test_size=0.3, random_state=156)


## 알고리즘 선택

### 선형회귀(LinearRegression)

In [42]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

# Linear Regression OLS로 학습/예측/평가 수행. 
lr = LinearRegression()
lr.fit(X_train ,y_train )
y_preds = lr.predict(X_test)
mse = mean_squared_error(y_test, y_preds)
rmse = np.sqrt(mse)

print('MSE : {0:.3f} , RMSE : {1:.3F}'.format(mse , rmse))
print('Variance score : {0:.3f}'.format(r2_score(y_test, y_preds)))

MSE : 31443.488 , RMSE : 177.323
Variance score : 0.908


In [43]:
print('절편 값:',lr.intercept_) # 독립 변수(X)가 모두 0일 때 종속 변수(y)의 예측값
print('회귀 계수값:', np.round(lr.coef_, 1)) # 종속 변수(y)에 대한 기여도

절편 값: 183.76415637296793
회귀 계수값: [ -12.1   -3.8 -462.9 2787.3  130.2   28.9   -4.9    4. ]


coef_ 속성은 회귀 계수 값만 가지고 있으므로 이를 피처별 회귀 계수 값으로 다시 매핑

In [44]:
# 회귀 계수를 큰 값 순으로 정렬하기 위해 Series로 생성. index가 칼럼명에 유의
coeff = pd.Series(data=np.round(lr.coef_, 1), index=X_data.columns )
coeff.sort_values(ascending=False) # 변수들의 중요도순으로

Total Revenue             2787.3
Internet Service           130.2
Device Protection Plan      28.9
Churn Score                  4.0
Under 30                    -3.8
Unlimited Data              -4.9
Dependents                 -12.1
Monthly Charge            -462.9
dtype: float64

* 교차검증

In [45]:
from sklearn.model_selection import cross_val_score

y_target = scaled_df['Contract Period']
X_data = scaled_df.drop(['Contract Period'],axis=1,inplace=False)
lr = LinearRegression()

# cross_val_score( )로 5 Fold 셋으로 MSE 를 구한 뒤 이를 기반으로 다시  RMSE 구함. 
neg_mse_scores = cross_val_score(lr, X_data, y_target, scoring="neg_mean_squared_error", cv = 5)
rmse_scores  = np.sqrt(-1 * neg_mse_scores)
avg_rmse = np.mean(rmse_scores)

# cross_val_score(scoring="neg_mean_squared_error")로 반환된 값은 모두 음수 
print(' 5 folds 의 개별 Negative MSE scores: ', np.round(neg_mse_scores, 2))
print(' 5 folds 의 개별 RMSE scores : ', np.round(rmse_scores, 2))
print(' 5 folds 의 평균 RMSE : {0:.3f} '.format(avg_rmse))

 5 folds 의 개별 Negative MSE scores:  [-29158.06 -29328.49 -24262.68 -32258.35 -37856.95]
 5 folds 의 개별 RMSE scores :  [170.76 171.26 155.76 179.61 194.57]
 5 folds 의 평균 RMSE : 174.390 


### 릿지 회귀(Ridge)

In [46]:
from sklearn.linear_model import Lasso, ElasticNet, Ridge

# alpha값에 따른 회귀 모델의 폴드 평균 RMSE를 출력하고 회귀 계수값들을 DataFrame으로 반환 
def get_linear_reg_eval(model_name, params=None, X_data_n=None, y_target_n=None, 
                        verbose=True, return_coeff=True):
    coeff_df = pd.DataFrame()
    if verbose : print('####### ', model_name , '#######')
    for param in params:
        if model_name =='Ridge': model = Ridge(alpha=param)
        elif model_name =='Lasso': model = Lasso(alpha=param)
        elif model_name =='ElasticNet': model = ElasticNet(alpha=param, l1_ratio=0.7)
        neg_mse_scores = cross_val_score(model, X_data_n, 
                                             y_target_n, scoring="neg_mean_squared_error", cv = 5)
        avg_rmse = np.mean(np.sqrt(-1 * neg_mse_scores))
        print('alpha {0}일 때 5 폴드 세트의 평균 RMSE: {1:.3f} '.format(param, avg_rmse))
        # cross_val_score는 evaluation metric만 반환하므로 모델을 다시 학습하여 회귀 계수 추출
        
        model.fit(X_data_n , y_target_n)
        if return_coeff:
            # alpha에 따른 피처별 회귀 계수를 Series로 변환하고 이를 DataFrame의 컬럼으로 추가. 
            coeff = pd.Series(data=model.coef_ , index=X_data_n.columns )
            colname='alpha:'+str(param)
            coeff_df[colname] = coeff
    
    return coeff_df
# end of get_linear_regre_eval

In [47]:
# 릿지에 사용될 alpha 파라미터의 값들을 정의하고 get_linear_reg_eval() 함수 호출
ridge_alphas = [0 , 0.1 , 1 , 10 , 100]
coeff_ridge_df =get_linear_reg_eval('Ridge', params=ridge_alphas, X_data_n=X_data, y_target_n=y_target)

#######  Ridge #######
alpha 0일 때 5 폴드 세트의 평균 RMSE: 174.390 
alpha 0.1일 때 5 폴드 세트의 평균 RMSE: 174.386 
alpha 1일 때 5 폴드 세트의 평균 RMSE: 174.680 
alpha 10일 때 5 폴드 세트의 평균 RMSE: 195.822 
alpha 100일 때 5 폴드 세트의 평균 RMSE: 370.933 


### 라쏘 회귀(Lasso)

In [48]:
# 라쏘에 사용될 alpha 파라미터의 값들을 정의하고 get_linear_reg_eval() 함수 호출
lasso_alphas = [ 0.07, 0.1, 0.5, 1, 3]
coeff_lasso_df =get_linear_reg_eval('Lasso', params=lasso_alphas, X_data_n=X_data, y_target_n=y_target)

#######  Lasso #######
alpha 0.07일 때 5 폴드 세트의 평균 RMSE: 174.374 
alpha 0.1일 때 5 폴드 세트의 평균 RMSE: 174.368 
alpha 0.5일 때 5 폴드 세트의 평균 RMSE: 174.336 
alpha 1일 때 5 폴드 세트의 평균 RMSE: 174.516 
alpha 3일 때 5 폴드 세트의 평균 RMSE: 177.469 


In [49]:
# 반환된 coeff_lasso_df를 첫번째 컬럼순으로 내림차순 정렬하여 회귀계수 DataFrame출력
sort_column = 'alpha:'+str(lasso_alphas[0])
coeff_lasso_df.sort_values(by=sort_column, ascending=False)

,alpha:0.07,alpha:0.1,alpha:0.5,alpha:1,alpha:3
Total Revenue,2772.928929,2771.461766,2751.906541,2727.573500,2630.723659
Internet Service,137.921301,136.142050,112.396923,83.880323,0.000000
Device Protection Plan,23.763667,23.601176,21.437677,18.799052,8.375254
Under 30,-2.841073,-2.594368,-0.000000,-0.000000,-0.000000
Churn Score,-8.688790,-8.278668,-2.829447,-0.000000,-0.000000
Unlimited Data,-11.820701,-11.383781,-5.560188,-0.000000,0.000000
Dependents,-17.758200,-17.098238,-8.292417,-0.000000,-0.000000
Monthly Charge,-435.368432,-433.243646,-404.887151,-369.883685,-240.435187


### 엘라스틱넷 회귀

In [50]:
# 엘라스틱넷에 사용될 alpha 파라미터의 값들을 정의하고 get_linear_reg_eval() 함수 호출
# l1_ratio는 0.7로 고정
elastic_alphas = [ 0.07, 0.1, 0.5, 1, 3]
coeff_elastic_df =get_linear_reg_eval('ElasticNet', params=elastic_alphas,
                                      X_data_n=X_data, y_target_n=y_target)

#######  ElasticNet #######
alpha 0.07일 때 5 폴드 세트의 평균 RMSE: 261.359 
alpha 0.1일 때 5 폴드 세트의 평균 RMSE: 292.789 
alpha 0.5일 때 5 폴드 세트의 평균 RMSE: 444.702 
alpha 1일 때 5 폴드 세트의 평균 RMSE: 495.289 
alpha 3일 때 5 폴드 세트의 평균 RMSE: 548.828 


In [51]:
# 반환된 coeff_elastic_df를 첫번째 컬럼순으로 내림차순 정렬하여 회귀계수 DataFrame출력
sort_column = 'alpha:'+str(elastic_alphas[0])
coeff_elastic_df.sort_values(by=sort_column, ascending=False)

,alpha:0.07,alpha:0.1,alpha:0.5,alpha:1,alpha:3
Total Revenue,1676.779316,1451.344919,554.617431,321.722787,121.458903
Device Protection Plan,122.314185,143.626221,181.892865,147.868648,75.643059
Monthly Charge,68.605823,123.795082,160.468618,114.440984,50.946653
Churn Score,18.457090,22.310354,21.737808,14.582124,5.059152
Under 30,-0.000000,-0.000000,-0.218789,-0.000000,-0.000000
Internet Service,-0.000000,-1.163484,20.905103,21.188016,11.686639
Dependents,-31.208137,-34.121069,-30.583082,-21.187712,-8.364383
Unlimited Data,-32.051176,-34.073978,-13.336285,-1.626439,0.733343


### 알고리즘 선택

선택 기준 : RMSE

* 선형회귀(LinearRegression) : 174.390  (cv=5)

* **릿지 회귀(Ridge) : 174.390 (alpha = 0.1, cv=5)**

* 라쏘 회귀(Lasso) : 174.336 (alpha = 0.07, cv=5)

* 엘라스틱넷 회귀(ElasticNet) : 261.359 (alpha = 0.07, cv=5)

# 알고리즘 적용하기

## 이탈하지 않은 고객(Churn Label=0)에게 적용

In [52]:
test_df = df.loc[df['Churn Label'] == 0]
test_df

,Customer ID,Dependents,Under 30,Monthly Charge,Total Revenue,Tenure in Months,Internet Service,Device Protection Plan,Unlimited Data,Streaming Movies,Contract Date,Contract Period,Churn Label,Churn Score
476,3841-NFECX,0,0,96.35,10105.37,71,1,1,1,0,2017-11-03,2157,0,38
477,4929-XIHVW,0,0,95.50,257.85,2,1,1,0,1,2023-08-06,55,0,69
478,3413-BMNZE,0,0,45.25,57.14,1,1,0,1,0,2023-08-18,43,0,40
479,8012-SOUDQ,0,0,90.25,4379.69,43,1,0,1,0,2020-03-23,1286,0,52
480,6575-SUVOI,0,0,69.50,2517.65,25,1,0,1,0,2021-09-12,748,0,25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,2569-WGERO,0,0,21.15,3039.53,72,0,0,0,0,2017-10-11,2180,0,45
7039,6840-RESVB,1,0,84.80,2807.47,24,1,1,1,1,2021-10-10,720,0,59
7040,2234-XADUH,1,0,103.20,9453.04,72,1,1,1,1,2017-10-14,2177,0,71
7041,4801-JZAZL,1,0,29.60,319.21,11,1,0,1,0,2022-11-28,306,0,59


In [56]:
# 알고리즘 적용할 컬럼 선택 (Customer ID와 label 컬럼 제외)
test_col = ['Dependents', 'Under 30', 'Monthly Charge','Total Revenue', 'Internet Service', 
            'Device Protection Plan','Unlimited Data', 'Churn Score']

new_test_df = test_df[test_col]
new_test_df

,Dependents,Under 30,Monthly Charge,Total Revenue,Internet Service,Device Protection Plan,Unlimited Data,Churn Score
476,0,0,96.35,10105.37,1,1,1,38
477,0,0,95.50,257.85,1,1,0,69
478,0,0,45.25,57.14,1,0,1,40
479,0,0,90.25,4379.69,1,0,1,52
480,0,0,69.50,2517.65,1,0,1,25
...,...,...,...,...,...,...,...,...
7038,0,0,21.15,3039.53,0,0,0,45
7039,1,0,84.80,2807.47,1,1,1,59
7040,1,0,103.20,9453.04,1,1,1,71
7041,1,0,29.60,319.21,1,0,1,59


* 정규화

In [57]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler

scaled_df_test = new_test_df.copy()

scaled_cols = ['Monthly Charge', 'Total Revenue', 'Churn Score']

# MinMaxScaler를 사용한 정규화
scaler = MinMaxScaler()
scaled_df_test[scaled_cols] = scaler.fit_transform(scaled_df_test[scaled_cols])
scaled_df_test

,Dependents,Under 30,Monthly Charge,Total Revenue,Internet Service,Device Protection Plan,Unlimited Data,Churn Score
476,0,0,0.777114,0.843287,1,1,1,0.440000
477,0,0,0.768657,0.019777,1,1,0,0.853333
478,0,0,0.268657,0.002992,1,0,1,0.466667
479,0,0,0.716418,0.364470,1,0,1,0.626667
480,0,0,0.509950,0.208755,1,0,1,0.266667
...,...,...,...,...,...,...,...,...
7038,0,0,0.028856,0.252398,0,0,0,0.533333
7039,1,0,0.662189,0.232992,1,1,1,0.720000
7040,1,0,0.845274,0.788735,1,1,1,0.880000
7041,1,0,0.112935,0.024908,1,0,1,0.720000


* 알고리즘 적용

In [58]:
from sklearn.linear_model import Ridge

# Ridge 회귀 모델 생성 및 학습 (이미 학습한 모델이라면 생략 가능)
ridge = Ridge(alpha=0.1)
ridge.fit(X_data, y_target)

# 새로운 데이터프레임 test_df를 이용하여 예측 수행
y_pred = ridge.predict(scaled_df_test)

# 예측 결과를 출력하거나 다른 작업에 활용
print(y_pred)

# 예측 결과를 "Contract Period"라는 컬럼으로 저장
test_df['Expected Period'] = y_pred.astype(int)

test_df

[2322.4413884    53.01633168  191.27050149 ... 2117.96605324  298.32981702
 1999.84368076]


C:\Users\ksh\AppData\Local\Temp\ipykernel_3272\2930271892.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['Expected Period'] = y_pred.astype(int)


,Customer ID,Dependents,Under 30,Monthly Charge,Total Revenue,Tenure in Months,Internet Service,Device Protection Plan,Unlimited Data,Streaming Movies,Contract Date,Contract Period,Churn Label,Churn Score,Expected Period
476,3841-NFECX,0,0,96.35,10105.37,71,1,1,1,0,2017-11-03,2157,0,38,2322
477,4929-XIHVW,0,0,95.50,257.85,2,1,1,0,1,2023-08-06,55,0,69,53
478,3413-BMNZE,0,0,45.25,57.14,1,1,0,1,0,2023-08-18,43,0,40,191
479,8012-SOUDQ,0,0,90.25,4379.69,43,1,0,1,0,2020-03-23,1286,0,52,995
480,6575-SUVOI,0,0,69.50,2517.65,25,1,0,1,0,2021-09-12,748,0,25,657
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,2569-WGERO,0,0,21.15,3039.53,72,0,0,0,0,2017-10-11,2180,0,45,858
7039,6840-RESVB,1,0,84.80,2807.47,24,1,1,1,1,2021-10-10,720,0,59,659
7040,2234-XADUH,1,0,103.20,9453.04,72,1,1,1,1,2017-10-14,2177,0,71,2117
7041,4801-JZAZL,1,0,29.60,319.21,11,1,0,1,0,2022-11-28,306,0,59,298


예측 결과인 Contract Period를 파생변수 Contract Data에서 더하여 예상 해지 날짜를 구한다.

In [59]:
from datetime import timedelta

# 'Contract Date' 컬럼을 datetime 형식으로 변환
test_df['Contract Date'] = pd.to_datetime(test_df['Contract Date'])

# 각 행에 대해 날짜 계산
test_df['Expected Date'] = test_df.apply(lambda row: row['Contract Date'] + timedelta(days=row['Expected Period']), axis=1)

# 결과 확인
test_df

C:\Users\ksh\AppData\Local\Temp\ipykernel_3272\86952396.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['Contract Date'] = pd.to_datetime(test_df['Contract Date'])
C:\Users\ksh\AppData\Local\Temp\ipykernel_3272\86952396.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['Expected Date'] = test_df.apply(lambda row: row['Contract Date'] + timedelta(days=row['Expected Period']), axis=1)


,Customer ID,Dependents,Under 30,Monthly Charge,Total Revenue,Tenure in Months,Internet Service,Device Protection Plan,Unlimited Data,Streaming Movies,Contract Date,Contract Period,Churn Label,Churn Score,Expected Period,Expected Date
476,3841-NFECX,0,0,96.35,10105.37,71,1,1,1,0,2017-11-03,2157,0,38,2322,2024-03-13
477,4929-XIHVW,0,0,95.50,257.85,2,1,1,0,1,2023-08-06,55,0,69,53,2023-09-28
478,3413-BMNZE,0,0,45.25,57.14,1,1,0,1,0,2023-08-18,43,0,40,191,2024-02-25
479,8012-SOUDQ,0,0,90.25,4379.69,43,1,0,1,0,2020-03-23,1286,0,52,995,2022-12-13
480,6575-SUVOI,0,0,69.50,2517.65,25,1,0,1,0,2021-09-12,748,0,25,657,2023-07-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,2569-WGERO,0,0,21.15,3039.53,72,0,0,0,0,2017-10-11,2180,0,45,858,2020-02-16
7039,6840-RESVB,1,0,84.80,2807.47,24,1,1,1,1,2021-10-10,720,0,59,659,2023-07-31
7040,2234-XADUH,1,0,103.20,9453.04,72,1,1,1,1,2017-10-14,2177,0,71,2117,2023-08-01
7041,4801-JZAZL,1,0,29.60,319.21,11,1,0,1,0,2022-11-28,306,0,59,298,2023-09-22


문제 : 예상 해지일(Expected Date)가 기준일인 2023년 9월 30일보다 이전인 경우가 있음.

In [60]:
# 'Expected Date'가 2023년 9월 30일보다 이전인 행 필터링
filtered_df = test_df[test_df['Expected Date'] < '2023-09-30']

# 결과 확인
filtered_df

,Customer ID,Dependents,Under 30,Monthly Charge,Total Revenue,Tenure in Months,Internet Service,Device Protection Plan,Unlimited Data,Streaming Movies,Contract Date,Contract Period,Churn Label,Churn Score,Expected Period,Expected Date
477,4929-XIHVW,0,0,95.50,257.85,2,1,1,0,1,2023-08-06,55,0,69,53,2023-09-28
479,8012-SOUDQ,0,0,90.25,4379.69,43,1,0,1,0,2020-03-23,1286,0,52,995,2022-12-13
480,6575-SUVOI,0,0,69.50,2517.65,25,1,0,1,0,2021-09-12,748,0,25,657,2023-07-01
481,4667-QONEA,0,0,74.85,7390.35,60,1,1,1,1,2018-10-08,1818,0,59,1784,2023-08-27
482,5067-XJQFU,0,0,108.45,8514.49,66,1,1,1,1,2018-04-02,2007,0,74,1896,2023-06-11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7037,7750-EYXWZ,0,0,60.65,702.89,12,1,1,1,1,2022-10-25,340,0,24,300,2023-08-21
7038,2569-WGERO,0,0,21.15,3039.53,72,0,0,0,0,2017-10-11,2180,0,45,858,2020-02-16
7039,6840-RESVB,1,0,84.80,2807.47,24,1,1,1,1,2021-10-10,720,0,59,659,2023-07-31
7040,2234-XADUH,1,0,103.20,9453.04,72,1,1,1,1,2017-10-14,2177,0,71,2117,2023-08-01


## 가상의 고객에게 적용

In [62]:
test_customer = [[0,0,0.716418,0.264570,1,0,0,0.466667]]

from sklearn.linear_model import Ridge

# Ridge 회귀 모델 생성 및 학습 (이미 학습한 모델이라면 생략 가능)
ridge = Ridge(alpha=0.1)
ridge.fit(X_data, y_target)

# 가상의 고객 데이터 이용하여 예측 수행
y_pred = ridge.predict(test_customer)
y_pred

C:\Users\ksh\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but Ridge was fitted with feature names
  warnings.warn(


array([733.3437158])